# Chapter 2: Corpus Processing Tools
Pierre Nugues

We use the `regex` module to have a better Unicode support

In [1]:
import regex as re

## Matching one occurrence

#### A first match with `re.search()`

In [2]:
line = 'The aerial acceleration alerted the ace pilot'
match = re.search('ab*c', line)
match      # <regex.Match object; span=(11, 13), match='ac'>

<regex.Match object; span=(11, 13), match='ac'>

#### Getting the match value

In [3]:
match.group() # ac

'ac'

## Getting all the matches

#### The list of all the strings

In [4]:
match_list = re.findall('ab*c', line)   # ['ac', 'ac']
match_list

['ac', 'ac']

#### The match groups (the objects)

In [5]:
match_iter = re.finditer('ab*c', line)   
list(match_iter)

[<regex.Match object; span=(11, 13), match='ac'>,
 <regex.Match object; span=(36, 38), match='ac'>]

## Interactive match

#### Using the shell (does not work with the notebooks)

In [6]:
import sys

for line in sys.stdin:
    if re.search('ab*c', line):    # m/ab*c/
        print('-> ' + line, end='')

#### Using IPython ipywidgets

In [7]:
# https://github.com/ipython/ipywidgets
import ipywidgets as widgets
from IPython.display import display

# The input box
text = widgets.Text()
display(text)

def handle_submit(sender):
    if re.search('ab*c', text.value):
        print('->', text.value)
    text.value = ''

# Hitting return fires handle_submit
text.on_submit(handle_submit)

Text(value='')

## Nonprintable characters and modifiers
#### Start of a line

In [8]:
# text = sys.stdin.read()
text = """Sing, O goddess, the anger of Achilles 
son of Peleus, that brought countless ills upon the Achaeans.
""".split('\n')
for line in text:
    match = re.search('^s', line) # m/^s/
    if match:
        print('-> ' + match.group())

-> s


#### The case-insensitive modifier

In [9]:
text = """Sing, O goddess, the anger of Achilles 
son of Peleus, that brought countless ills upon the Achaeans.
""".split('\n')
for line in text:
    match = re.search('^s', line, re.I) # m/^s/i
    if match:
        print('-> ' + match.group())

-> S
-> s


#### Case insensitive and multiline: A `\n` defines a new start of string 

In [10]:
text = """Sing, O goddess, the anger of Achilles son
of Peleus, that brought countless ills upon the Achaeans.
"""
match = re.search('^s', text, re.I | re.M) # m/^s/im
if match:
    print('-> ' + match.group())
match = re.search('^o', text, re.I | re.M)
if match:
    print('-> ' + match.group())

-> S
-> o


#### Getting all the matches

In [11]:
text = """Sing, O goddess, the anger of Achilles 
son of Peleus, that brought countless ills upon the Achaeans.
"""
match_list = re.findall('^s', text, re.I | re.M)
print(match_list)

['S', 's']


#### Getting all the matches with `finditer()`

In [12]:
text = """Sing, O goddess, the anger of Achilles 
son of Peleus, that brought countless ills upon the Achaeans.
"""
match_list = re.finditer('^s', text, re.I | re.M)
print(match_list)
for match in match_list:
    print(match)

<regex.Match object; span=(0, 1), match='S'>
<regex.Match object; span=(40, 41), match='s'>


## Substitution

#### Global replacement: `s/regex/replacement/g`

In [13]:
text = """Sing, O goddess, the anger of Achilles 
son of Peleus, that brought countless ills upon the Achaeans.
""".split('\n')
for line in text:
    if re.search('es+', line):
        print("Old: " + line)
        # Replaces all the occurrences
        line = re.sub('es+', 'ES', line)
        print("New: " + line)
# s/ab+c/ABC/g

Old: Sing, O goddess, the anger of Achilles 
New: Sing, O goddES, the anger of AchillES 
Old: son of Peleus, that brought countless ills upon the Achaeans.
New: son of Peleus, that brought countlES ills upon the Achaeans.


 
#### Just one replacement: s/regex/replacement/

In [14]:
text = """Sing, O goddess, the anger of Achilles 
son of Peleus, that brought countless ills upon the Achaeans.
""".split('\n')
for line in text:
    if re.search('es+', line):
        print("Old: " + line)
        # Replaces all the occurrences
        line = re.sub('es+', 'ES', line, 1)
        print("New: " + line)
# s/ab+c/ABC/


Old: Sing, O goddess, the anger of Achilles 
New: Sing, O goddES, the anger of Achilles 
Old: son of Peleus, that brought countless ills upon the Achaeans.
New: son of Peleus, that brought countlES ills upon the Achaeans.


## Backreferences

In [15]:
line = 'abbbcdeeef'
match = re.search(r'(.)\1\1', line)
match.group(1)                # 'b'

'b'

 #### Substitutions `s/(.)\1\1/***/g`

In [16]:
re.sub(r'(.)\1\1', '***', 'abbbcdeeef')  # 'a***cd***f'

'a***cd***f'

#### Multiple backreferences `m/\$ *([0-9]+)\.?([0-9]*)/`

In [17]:
price = "We'll buy it for $72.40"
match = re.search('\$ *([0-9]+)\.?([0-9]*)', price)
match.group() # ’$72.40’ The entire match

'$72.40'

In [18]:
match.group(1) # ’72’ The first group

'72'

In [19]:
match.group(2) # ’40’ The second group

'40'

#### Substitutions `s/\$ *([0-9]+)\.?([0-9]*)/\1 dollars and \2 cents/g`

In [20]:
price = "We'll buy it for $72.40"
re.sub('\$ *([0-9]+)\.?([0-9]*)',
       r'\1 dollars and \2 cents', price)
   # We’ll buy it for 72 dollars and 40 cents

"We'll buy it for 72 dollars and 40 cents"

#### Why `r`

In [21]:
'\1'

'\x01'

In [22]:
print('\141')

a


In [23]:
r'\1'

'\\1'

In [24]:
print(r'\141')

\141


## Matching objects

In [25]:
price = "We'll buy it for $72.40"
match = re.search('\$ *([0-9]+)\.?([0-9]*)', price)

#### Input

In [26]:
match.string            # We’ll buy it for $72.40

"We'll buy it for $72.40"

#### Groups

In [27]:
match.groups(0)          # (’72’, ’40’)

('72', '40')

In [28]:
match.group(0)          # '$72.40'

'$72.40'

In [29]:
match.group(1)

'72'

#### Match objects: The indices

In [30]:
match.start(0)

17

In [31]:
match.end(0)

23

In [32]:
match.start(1)

18

In [33]:
match.end(1)

20

#### Example

In [34]:
line = """Tell me, O muse, of that ingenious hero
  who travelled far and wide after he had sacked
  the famous town of Troy."""
match = re.search(',.*,', line, re.S)
line[0:match.start()]             # ’Tell me’

'Tell me'

In [35]:
line[match.start():match.end()]   # ’, O muse,’

', O muse,'

In [36]:
line[match.end():]   # ’of that ingenious hero
         #  who travelled far and wide after he had sacked
         #  the famous town of Troy.’

' of that ingenious hero\n  who travelled far and wide after he had sacked\n  the famous town of Troy.'

## Concordances: `.{0,15}Nils Holgersson.{0,15}`

In [37]:
pattern = 'Nils Holgersson'
width = 15
# .{0,width}pattern.{0,width}

('.{{0,{width}}}{pattern}.{{0,{width}}}'
 .format(pattern=pattern, width=width))

'.{0,15}Nils Holgersson.{0,15}'

In [38]:
file_name = '../../corpus/Selma.txt'
text = open(file_name).read()
text[:20]

'Nils Holgerssons und'

In [39]:
# spaces match tabs and newlines
pattern = re.sub(' ', '\\s+', pattern)
# Replaces newlines with spaces in the text
text = re.sub('\s+', ' ', text)
concordance = ('(.{{0,{width}}}{pattern}.{{0,{width}}})'
               .format(pattern=pattern, width=width))
for match in re.finditer(concordance, text):
    print(match.group(1))

Nils Holgerssons underbara res
mmetott! Se på Nils Holgersson Tummetott!" Ge
an. "Jag heter Nils Holgersson och är son til
e är det värt, Nils Holgersson, att du är äng
 den tiden, då Nils Holgersson drog omkring m
visa honom vad Nils Holgersson från Västra Ve
m det året, då Nils Holgersson for omkring me
kan kosta dem. Nils Holgersson hade inte haft
e mer sägas om Nils Holgersson, att han inte 
" För där stod Nils Holgersson mitt uppe på R
ingo de syn på Nils Holgersson, och då sköt d
vildgässen och Nils Holgersson äntligen hade 
etare. Men vad Nils Holgersson inte så, det v
åga, och om då Nils Holgersson sade nej, börj
ats, och om nu Nils Holgersson också hade teg
mädlig ut, att Nils Holgersson kastade sig öv
Och inte ville Nils Holgersson slåss med en t
den tiden, när Nils Holgersson for omkring me
kull honom. Om Nils Holgersson genast hade ro
 på egen hand, Nils Holgersson," sade han då 
mle-Drumle ner Nils Holgersson på bottnen av 
 - "Jo, jag är Nils Holgersson från Västra Ve
 de